In [1]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import re

In [73]:
"""
Input for the code:
    $ A fasta file containing your DNA sequence that you want to edit with gRNAs (eg. pXW487, in fasta format)
    $ A text file containing two columns (no header),
        one for SNP names (eg rs21062170),
        the other for the corresponding SNP locations (the number should point to the SNPs in the fasta file)
"""

SEQ_FILENAME = 'pXW487.fasta'
SNP_FILENAME = 'SNPs.txt'
OUTPUT_FILENAME = "grna_pam_strand.txt"

desktop = os.path.expanduser("~/Desktop")

seq_file_path = desktop + '/' + SEQ_FILENAME
snp_file_path = desktop + '/' + SNP_FILENAME
output_file_path = desktop + '/' + OUTPUT_FILENAME

In [3]:
# Load the fasta sequence from the file
def load_fasta_sequence(file_path):
    with open(file_path, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            return str(record.seq)

In [4]:
my_sequence = load_fasta_sequence(seq_file_path)

In [51]:
# Load the SNP file
def load_snps(file_path):
    df = pd.read_csv(file_path, header=None, sep='\t').dropna(axis=1, how='all')
    if df.shape[1] != 2:
        print("Error: column number is not 2")
    else:
        if ('rs' or 'Rs') in df[0][0] or int(df.iloc[0,1].replace(",", "")):
            df.columns = ['snp', 'loc']
        else:
            df.columns = ['loc', 'snp']
    df["loc"] = df["loc"].str.replace(",", "").astype(int)
    return df

In [52]:
snp_df = load_snps(snp_file_path)
snp_df

,snp,loc
0,rs10774035,82433
1,rs10774036,100707
2,rs10744560,100858
3,rs12311439,108540
4,rs1024582,116005
5,rs4298967,121953


In [71]:
def find_nearest_pam(sequence, snp_pos, neighbor_window=50):
    neighbor_seq = sequence[snp_pos - neighbor_window:snp_pos + neighbor_window]
    distances_to_50 = []

    pam_patterns = [re.compile("(?=GG)"), re.compile("(?=CC)")]
    strand_labels = ['+', '-']

    for pam_pattern, strand_label in zip(pam_patterns, strand_labels):
        for match in pam_pattern.finditer(neighbor_seq.replace("g","G").replace("c","C")):
            if strand_label == "+":
                pam_pos = match.start() - 1
                cut_site_pos = pam_pos - 3
            else:
                pam_pos = match.start()
                cut_site_pos = pam_pos + 6
            distance_to_50 = abs(cut_site_pos - 50)
            distances_to_50.append((distance_to_50, pam_pos, strand_label))

    distances_to_50.sort(key=lambda x: x[0])  # Sort by distance

    closest_distance_to_50, closest_pam_pos, closest_strand = distances_to_50[0]

    if closest_strand == '+':
        closest_gRNA_sequence = neighbor_seq[closest_pam_pos - 20: closest_pam_pos]
        closest_pam_seq = neighbor_seq[closest_pam_pos:closest_pam_pos + 3]
    else:
        closest_gRNA_sequence = str(Seq(neighbor_seq[closest_pam_pos + 3: closest_pam_pos + 23]).reverse_complement())
        closest_pam_seq = str(Seq(neighbor_seq[closest_pam_pos:closest_pam_pos + 3]).reverse_complement())


    print("Closest NGG or CCN Position:", closest_pam_pos )
    print("Closest Distance to Cut Site (3 nt upstream of PAM) from SNP:", closest_distance_to_50)
    print("Closest gRNA Sequence:", closest_gRNA_sequence)
    print("closest PAM", closest_pam_seq)
    print("Closest Strand:", closest_strand)


    return closest_gRNA_sequence, closest_pam_seq, closest_strand


In [78]:
gRNAs = []
pams = []
strands = []

for index, row in snp_df.iterrows():
    snp_position = row["loc"]
    snp_name = row["snp"]
    print("\n" + snp_name)
    gRNA, pam, strand = find_nearest_pam(my_sequence,snp_position)
    gRNAs.append(gRNA)
    pams.append(pam)
    strands.append(strand)

# Save the gRNAs, PAMs, and strands in a text file

with open(output_file_path, "w") as file:
    for snp_name, gRNA, pam, strand in zip(snp_df["snp"], gRNAs, pams, strands):
        file.write(f"{snp_name}\tgRNA: {gRNA}\tPAM: {pam}\tStrand: {strand}\n")

print("Data saved to", output_file_path)


rs10774035 
Closest NGG or CCN Position: 48
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 4
Closest gRNA Sequence: CCCCTTGCAACATCTCAAAG
closest PAM CGG
Closest Strand: -

rs10774036
Closest NGG or CCN Position: 42
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 2
Closest gRNA Sequence: CAATTTTTGGTGAGCGGTTA
closest PAM TGG
Closest Strand: -

rs10744560
Closest NGG or CCN Position: 42
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 2
Closest gRNA Sequence: CTCAGCACGATGAGTGAGAC
closest PAM TGG
Closest Strand: -

rs12311439
Closest NGG or CCN Position: 46
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 7
Closest gRNA Sequence: ACTCTGCTGTGCCGCAGAGC
closest PAM TGG
Closest Strand: +

rs1024582
Closest NGG or CCN Position: 46
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 7
Closest gRNA Sequence: GTATTATTTGGTTGTTCACG
closest PAM GGG
Closest Strand: +

rs4298967
Closest NGG or CCN Position: 39
Closest Distance to Cut Sit

In [80]:
gRNAs

['CCCCTTGCAACATCTCAAAG',
 'CAATTTTTGGTGAGCGGTTA',
 'CTCAGCACGATGAGTGAGAC',
 'ACTCTGCTGTGCCGCAGAGC',
 'GTATTATTTGGTTGTTCACG',
 'AGGCTGAACGGAGCTAGTGA']

In [82]:

pams

['CGG', 'TGG', 'TGG', 'TGG', 'GGG', 'GGG']